In [1]:
import qubovert as qv
from qubovert import boolean_var
import pickle

def make_h_new(h_pickled):
    h_new = qv.PCBO()
    objective_lambda = 0.1

    def build_term_from_constraint(constraint):
        term = qv.PUBO()
        for pair_bools, coefficient in constraint.items():
            if len(pair_bools) == 0:
                term += coefficient
            elif len(pair_bools) == 1:
                term += boolean_var(pair_bools[0]) * coefficient
            else:
                term += boolean_var(pair_bools[0]) * boolean_var(pair_bools[1]) * coefficient
        return term

    # ----- Add less than constraints and objective -----
    for constraint in h_pickled['constraints']['lt']:
        left_term = build_term_from_constraint(constraint)
        h_new.add_constraint_lt_zero(left_term)
    #     ---- In this problem, the objective is the same as 0.1 * h['constraints']['lt']
        h_new += objective_lambda * left_term

    # ----- Add equal zero constraints -----
    for constraint in h_pickled['constraints']['eq']:
        left_term = build_term_from_constraint(constraint)
        h_new.add_constraint_eq_zero(left_term)
    assert h_new.to_qubo().Q == h_pickled['qubo']
    return h_new

with open('pickles/302-5780-7-31-sum_taus-2-7-232034-H.pickle', 'rb') as f:
    h_pickled = pickle.load(f)
h_new = make_h_new(h_pickled)

In [2]:
from neal import SimulatedAnnealingSampler
from dimod import BinaryQuadraticModel, Vartype
from functools import partial
import time
import wandb

NUM_READS = 1000

def time_execution(func, label='sampling_time'):
    execution_start = time.time()
    res = func()
    print(label, time.time() - execution_start)
    return res

def solve_using_s_annealing(h_new):
    qv_qubo = h_new.to_qubo()
    dwave_qubo = qv_qubo.Q
    sim_sampler = SimulatedAnnealingSampler()
    res = time_execution(partial(sim_sampler.sample_qubo, dwave_qubo, num_reads=NUM_READS))
#     print('solution energy', res.first.energy)
    qubo_solution = res.first.sample
    H_solution = h_new.convert_solution(qubo_solution)
    print("Constraints satisfied?", h_new.is_solution_valid(H_solution))
    print('h_new.value', h_new.value(H_solution))
    return H_solution


H_solution = solve_using_s_annealing(h_new)

sampling_time 9.73518681526184
Constraints satisfied? True
h_new.value -0.09999999999990905


In [6]:
def constarints_penalty(solution):
    print("Constraints satisfied?", h_new.is_solution_valid(solution))
    print('h_new.value', h_new.value(solution))
    lt_val = h_new.constraints['lt'][0].value(solution)
    if lt_val >= 0:
        print('lt value', lt_val)

    for constraint in h_new.constraints['eq']:
        val_ = constraint.value(solution)
        if val_ != 0:
            print('eq value', val_, 'constraint', constraint)

constarints_penalty(H_solution)

Constraints satisfied? True
h_new.value -0.09999999999990905


## Fujitsu Solution

In [120]:
import glob
for solution_path in glob.glob(dir_path + "/*.pickle"):
    file_name = Path(solution_path).name
    print(file_name)
    
    h_path = "pickles/" + file_name.replace('_solution', '')
    with open(h_path, 'rb') as f:
        h_dict = pickle.load(f)
    h_new = make_h_new(h_dict)
    dir_path = "Fujitsu-UCSB-master/results"
    with open(solution_path, 'rb') as f:
        fujitsu_solution = pickle.load(f)
    
    fujitsu_h_solution = h_new.convert_solution(fujitsu_solution)
    print("Constraints satisfied?", h_new.is_solution_valid(fujitsu_h_solution))
    print('h_new.value', h_new.value(fujitsu_h_solution))
    print("-----------------------")

9157-100677-7-31-sum_taus-32-15-237864-H_solution.pickle
Constraints satisfied? False
h_new.value 199.0
-----------------------
2317-29547-7-31-sum_taus-8-15-812112-H_solution.pickle
Constraints satisfied? False
h_new.value 19.900000000000546
-----------------------
607-11792-7-31-sum_taus-2-15-960004-H_solution.pickle
Constraints satisfied? True
h_new.value -0.3999999999996362
-----------------------
1177-17686-7-31-sum_taus-4-15-988191-H_solution.pickle
Constraints satisfied? False
h_new.value 28.699999999999818
-----------------------
34696-657777-7-31-sum_taus-32-31-10075-H_solution.pickle
Constraints satisfied? False
h_new.value 1243.7999999999993
-----------------------
50-322-4-7-sum_taus-2-3-113261-H_solution.pickle
Constraints satisfied? False
h_new.value 0.2999999999999545
-----------------------
302-5780-7-31-sum_taus-2-7-232034-H_solution.pickle
Constraints satisfied? True
h_new.value -0.5
-----------------------
